In [1]:
#| default_exp responseKeys

%pip install -q nbdev
%pip install -q pydantic

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# TODO: create response keys from scratch

We take the keys extracted from previous month's job listings and manually add a description. This too, could have been parsed by an LLM, but since the list is relatively short, and needs to be vetted, it's faster and more accurate to do this by hand, rather than creating a prompt flow and evals etc.

In [2]:
#| export

# These keys were retreived from previous analysis. The prompts will add any that are needed.
initial_keys = [
    'company: name of the company',
    'location: country, city or location otherwise stated',
    'timezone: timezones that are referred to in the job post, if any',
    'industry: the industry vertical that the company is part of',
    'target_market: the type of consumer or businesses that are targeted by the company venture',
    'company_status: the current situation that company is in',
    'company_description: description of the company, as described in the job post',
    'company_goal: primary focus of the company',
    'company_stage: venture stage of the company, if applicable',
    'more_company_info: an array. any company information in the listing not otherwise covered by company_goal, company_status, company_stage, company_description, industry or location',
    'employment_type: full-time, part-time, contract, or as otherwise described',
    'remote_or_local_details: details about the remote, local, hybrid or otherwise work arrangement as described',
    'job_title: job title for the position. Use the singular tense',
    'job_description: description of the job as described in the job post',
    'job_requirements: an array of the specific job requirements, listed in the job posting',
    'job_soft_skills: an array of any soft skills required by the job, as outlined in the posting',
    'product_description: a description of the product that job is related to, if any',
    'tech_stack: an array. It is list of the names of technologies used by the company. This can include programming languages, frameworks, libraries, databases, and other tools.',
    'application_process: any description of the application processes that the person applying will undergo',
    'contact_email: a single email. the contact email provided in the job post. if multiple emails are presented, add the additional as an item in the additional notes array',
    'application_url: a single url. The url used to apply for the job. information about the company other than the job listing should be placed in the information_urls key',
    'information_urls: an array. if the posting states that more information about the company or the job is available at a URL other than the job posting, recored those urls in the array here',
    'additional_notes: an array. any additional information about the company, role or position, not covered by the other keys',
    'compensation: any compensation and/or benefits described in the job post'
]

We export this as a string, for convenient inclusion in prompts.

In [3]:
#| export
keyStr = '\n'.join(initial_keys)


In [12]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

def getResponse(prompt):
    return model.invoke(prompt)

We use pydantic to create a Response class that we can use for our first layer of prompt evaluation. 

In [5]:
#| export
from pydantic import BaseModel, ValidationError
from langchain_core.prompts import PromptTemplate
import json

class Response(BaseModel):
    company: str = ''
    location: str = ''
    timezone: str = ''
    industry: str = ''
    target_market: str = ''
    company_status: str = ''
    company_description: str = ''
    company_goal: str = ''
    company_stage: str = ''
    more_company_info: list[str] = []
    employment_type: str = ''
    remote_local_details: str = ''
    job_title: str = ''
    job_description: str = ''
    job_requirements: list[str] = []
    job_soft_skills: list[str] = []
    product_description: str = ''
    tech_stack: list[str] = []
    application_process: str = ''
    contact_email: str = ''
    application_url: str = ''
    information_urls: list[str] = []
    additional_notes: list[str] = []
    compensation: str = ''

def callValidator(response):
    return Response(**response)

keyStr = "List of all possible keys and their descriptions"  # Replace with actual key descriptions

def validate(response, repromptNum=None):
    try:        
        validated_response = callValidator(response)
        print("Validation successful")
        return validated_response
    except ValidationError as e:
        if repromptNum is None:
            repromptNum = 0

        print('Validation error:', e)
        print('Attempt:', repromptNum + 1)

        if repromptNum > 3:
            return {'error': "Could not receive valid json: Too many reprompts", 'item': response}
        
        repromptNum += 1
        return repromptRevalidate(response, str(e), repromptNum)

def repromptRevalidate(item, error_message, repromptNum):
    promptTemplate = PromptTemplate(template='''
    Your response did not follow the instructions in regards to appropriate keys. 
    Here is the error message from pydantic:

    {error}

    Here is a description of the keys:

    {keyStr}

    Please update your response, fixing any mistakenly assigned keys. If information has been assigned to the wrong key, assign it to the right key. Follow the directions in regards to the keys exactly. 
    The output should be a single JSON object. Here is your response again:

    {item}
    ''', input_variables=["error", "keyStr", "item"])

    newPrompt = promptTemplate.format(error=error_message, keyStr=keyStr, item=json.dumps(item, indent=2))
    response = getResponse(newPrompt)

    print('Received response from AI')
    try:
        parsed_response = json.loads(response.content)
        return validate(parsed_response, repromptNum=repromptNum)
    except json.JSONDecodeError:
        print("Failed to parse JSON from response")
        return {'error': "Invalid JSON response", 'item': response.content}

# Test data
exampleOne = {
    "company": "Irth Solutions",
    "job_title": "Senior Software Engineer",
    "job_description": "Build software that improves resilience and reduce risk to critical infrastructure that millions of people and businesses rely on each day. Systems handle millions of requests a day, coordinating activities between critical infrastructure providers, contract locators, excavators, and other stakeholders throughout North America.",
    "tech_stack": "C# development experience using .NET Core, WebAPI, and Entity Framework. Experience building browser applications using Angular and Typescript. Strong relational database experience with Postgres (preferred) or MS SQL Server including SQL, database functions, indexing, and performance tuning. Kubernetes, Docker, Linux.",
    "contact_url": "https://apply.workable.com/irth-solutions/j/33AD66BA3A/"
}

# # Test the function
# result = validate(exampleOne)
# print(result)

Validation error: 1 validation error for Response
tech_stack
  Input should be a valid list [type=list_type, input_value='C# development experienc...ernetes, Docker, Linux.', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/list_type
Attempt: 1
Received response from AI
Validation successful
company='Irth Solutions' location='' timezone='' industry='' target_market='' company_status='' company_description='' company_goal='' company_stage='' more_company_info=[] employment_type='' remote_local_details='' job_title='Senior Software Engineer' job_description='Build software that improves resilience and reduce risk to critical infrastructure that millions of people and businesses rely on each day. Systems handle millions of requests a day, coordinating activities between critical infrastructure providers, contract locators, excavators, and other stakeholders throughout North America.' job_requirements=[] job_soft_skills=[] product_description='' tech_stac

In [14]:
#| export
from langchain_core.prompts import PromptTemplate

# Check each god damned key for discrepancies.

# keys = '''
# job_title: job title for the position. Use the singular tense
# job_description: description of the job as described in the job post
# tech_stack: an array. It is list of the names of technologies used by the company. It is not a list of descriptions. Do not return a list of phrases. This is not the same as job requirements list, use that key if the tech stack is not explicitly presented
# '''    

# promptTemplate = PromptTemplate(template='''Are all the keys on this item: {item} 
# either an empty string, or match the requirements in this list? 
# {keys}  If so, return the item. If not, return the item with the keys updated to match the requirements on the list
# ''')

# item = { 
#     'job_title': 'programmer',
#     'job_description': 'a programmer of stuff',
#     'tech_stack': result.model_dump()['tech_stack']
# }

# prompt = promptTemplate.format(item=result.model_dump(), keys=keyStr)

# print(prompt)
# response = getResponse(prompt)    

# response.content
    

In [8]:
# print(response.content)

In [15]:
#| hide
from nbdev import nbdev_export
nbdev_export()